#RED MPL

Entrenamiento

In [1]:
# entrenamiento.py
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import time
import psutil  # Para uso de recursos (CPU y Memoria)

# Definir el modelo MLP
def crear_modelo():
    modelo = Sequential()
    modelo.add(Dense(64, input_dim=3, activation='relu'))  # 3 características (promedio de R, G, B)
    modelo.add(Dense(32, activation='relu'))
    modelo.add(Dense(3, activation='softmax'))  # 3 clases: rojo, amarillo, verde
    modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return modelo

# Generar datos de entrenamiento
def generar_datos(n=1000):
    X_train = []
    y_train = []

    for _ in range(n):
        R = np.random.randint(0, 256)
        G = np.random.randint(0, 256)
        B = np.random.randint(0, 256)

        X_train.append([R, G, B])

        if R > G and R > B:
            y_train.append(0)  # Rojo
        elif G > R and G > B:
            y_train.append(1)  # Amarillo
        else:
            y_train.append(2)  # Verde

    return np.array(X_train), np.array(y_train)

# Función para evaluar precisión y tasa de error en los datos de prueba
def evaluar_precision(modelo, X_test, y_test):
    predicciones = modelo.predict(X_test)
    y_pred = np.argmax(predicciones, axis=1)
    y_true = np.argmax(y_test, axis=1)

    correctas = np.sum(y_pred == y_true)
    total = len(y_true)
    precision = correctas / total
    tasa_error = 1 - precision

    return precision, tasa_error

# Entrenar la red neuronal con los datos generados
def entrenar_red():
    # Generar conjunto de datos
    X_train, y_train = generar_datos(10000)  # 10,000 ejemplos

    encoder = LabelEncoder()
    y_train_encoded = encoder.fit_transform(y_train)
    y_train_onehot = tf.keras.utils.to_categorical(y_train_encoded, 3)

    # Dividir datos en entrenamiento y validación
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_onehot, test_size=0.2, random_state=42)

    # Crear modelo
    modelo = crear_modelo()

    # Medir tiempo de entrenamiento
    start_time = time.time()
    modelo.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)
    end_time = time.time()

    # Tiempo de entrenamiento
    tiempo_entrenamiento = end_time - start_time
    print(f"Tiempo de entrenamiento: {tiempo_entrenamiento:.2f} segundos")

    # Evaluar precisión y tasa de error en datos de validación
    precision, tasa_error = evaluar_precision(modelo, X_val, y_val)
    print(f"Precisión en datos de validación: {precision * 100:.2f}%")
    print(f"Tasa de error en datos de validación: {tasa_error * 100:.2f}%")

    # Guardar el modelo entrenado
    modelo.save("modelo_semaforo.h5")
    print("Modelo guardado como 'modelo_semaforo.h5'")

if __name__ == "__main__":
    entrenar_red()


C:\Users\Equipo\PycharmProjects\JupyterProject\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8324 - loss: 3.8690 - val_accuracy: 0.9840 - val_loss: 0.0757
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9746 - loss: 0.0884 - val_accuracy: 0.9525 - val_loss: 0.2234
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9723 - loss: 0.0856 - val_accuracy: 0.9640 - val_loss: 0.1126
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9754 - loss: 0.0743 - val_accuracy: 0.9860 - val_loss: 0.0457
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9742 - loss: 0.0796 - val_accuracy: 0.9610 - val_loss: 0.1268
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9698 - loss: 0.0917 - val_accuracy: 0.9940 - val_loss: 0.0320
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9826 - loss: 0.0568 - val_accuracy: 0.9540 - val_loss: 0.2053
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9745 - loss: 0.0883 - val_accuracy: 0.

Precisión en datos de validación: 99.00%
Tasa de error en datos de validación: 1.00%
Modelo guardado como 'modelo_semaforo.h5'


ejecucion

In [2]:
# prediccion.py
import cv2
import numpy as np
import tensorflow as tf
import time
import psutil  # Para uso de recursos (CPU y Memoria)

# Cargar el modelo previamente entrenado
modelo = tf.keras.models.load_model("entrenamientos_guardados/modelo_semaforo.h5")

# Preprocesar la imagen para extraer los colores predominantes
def obtener_color_promedio(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    rango_rojo = ((0, 100, 100), (10, 255, 255))
    rango_amarillo = ((20, 100, 100), (40, 255, 255))
    rango_verde = ((40, 100, 100), (80, 255, 255))

    rojo = cv2.inRange(hsv, rango_rojo[0], rango_rojo[1])
    amarillo = cv2.inRange(hsv, rango_amarillo[0], rango_amarillo[1])
    verde = cv2.inRange(hsv, rango_verde[0], rango_verde[1])

    promedio_rojo = cv2.countNonZero(rojo)
    promedio_amarillo = cv2.countNonZero(amarillo)
    promedio_verde = cv2.countNonZero(verde)

    return np.array([promedio_rojo, promedio_amarillo, promedio_verde])

# Función para predecir el color del semáforo en tiempo real
def predecir_color(frame):
    promedio_colores = obtener_color_promedio(frame)
    start_time = time.time()
    prediccion = modelo.predict(np.array([promedio_colores]), verbose=0)
    end_time = time.time()

    # Medir tiempo de ejecución
    tiempo_ejecucion = (end_time - start_time) * 1000  # en milisegundos

    clase_predicha = np.argmax(prediccion)

    if clase_predicha == 0:
        color_predicho = "Semaforo: Rojo - Detengase"
    elif clase_predicha == 1:
        color_predicho = "Semaforo: Amarillo - Precaucion"
    elif clase_predicha == 2:
        color_predicho = "Semaforo: Verde - Puede avanzar"

    return color_predicho, tiempo_ejecucion

# Iniciar la cámara y usar el modelo para predecir el color del semáforo
def iniciarCamara():
    print("Iniciando módulo de cámara...")
    cam = cv2.VideoCapture(0)

    if not cam.isOpened():
        print("No se pudo abrir la cámara")
        return

    print("Cámara abierta")

    predicciones_correctas = 0
    total_predicciones = 0

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Error al capturar imagen")
            break

        color_predicho, tiempo_ejecucion = predecir_color(frame)

        # Actualizar estadísticas
        total_predicciones += 1
        if color_predicho == "Semaforo: Rojo - Detengase":  # Suponiendo que sea el color correcto
            predicciones_correctas += 1

        precision = predicciones_correctas / total_predicciones
        print(f"Precisión: {precision * 100:.2f}%")
        print(f"Tiempo de ejecución (frame): {tiempo_ejecucion:.2f} ms")

        # Mostrar información en la pantalla
        cv2.putText(frame, f"Color: {color_predicho}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Tiempo: {tiempo_ejecucion:.2f} ms", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Camara", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # ESC
            break

    cam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    iniciarCamara()


Iniciando módulo de cámara...
Cámara abierta
Precisión: 0.00%
Tiempo de ejecución (frame): 176.53 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 81.78 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 80.78 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 74.50 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 83.55 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 75.04 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 83.78 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 91.75 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 89.76 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 126.66 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 80.78 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 88.76 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 84.77 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 78.79 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 90.76 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 78.79 ms
Precisión: 0.00%
Tiempo de ejecución (frame): 77.79 ms
Precisión: 0.00%
T